# 🎯 Interactive Robot Control

คลิกเลือกจุดจากภาพกล้อง → Robot เคลื่อนที่ไปยังจุดที่เลือก

**Features:**
- คลิกเลือกจุดจากภาพกล้อง
- แปลง pixel → robot coordinates ด้วย Homography Matrix
- Robot เคลื่อนที่ไปยังจุดที่เลือกอัตโนมัติ

## 1️⃣ Imports & Setup

In [ ]:
!pip install pydobot matplotlib opencv-python numpy

In [1]:
import cv2
import numpy as np
import time
import socket
import importlib

import config
importlib.reload(config)

print("✓ Imports loaded")

[Config] Mode: ROBOT, Capture: FRAME_BY_FRAME, Device: cpu
[Config] Mode: ROBOT, Capture: FRAME_BY_FRAME, Device: cpu
✓ Imports loaded


## 2️⃣ DobotControllerTCP Class

In [4]:
class DobotControllerTCP:
    def __init__(self, homography_matrix=None):
        self.dashboard_port = 29999
        self.sock_dashboard = None
        self.homography_matrix = homography_matrix
        self.grasp_height = config.ROBOT_GRASP_HEIGHT
        self.safe_height = config.ROBOT_SAFE_HEIGHT
        
    def connect(self, ip):
        try:
            self.sock_dashboard = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock_dashboard.settimeout(5)
            self.sock_dashboard.connect((ip, self.dashboard_port))
            
            print("Clearing Errors...")
            self.send_command("ClearError()")
            time.sleep(0.5)
            
            print("Enabling Robot...")
            self.send_command("EnableRobot()")
            time.sleep(4)
            
            # ใช้ User(1), Tool(1) ให้ตรงกับ Online mode
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            
            print("\n✅ Connected!")
            return True
        except Exception as e:
            print(f"Connection Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock_dashboard:
            try:
                self.sock_dashboard.send((cmd + "\n").encode("utf-8"))
                return self.sock_dashboard.recv(1024).decode("utf-8")
            except Exception as e:
                print(f"Command Error: {e}")
        return None

    def home(self):
        print("🤖 Moving to HOME...")
        self.send_command("JointMovJ(0,0,0,0)")
        time.sleep(4)
        print("✅ HOME")

    def move_to(self, x, y, z, r=0):
        """เคลื่อนที่ไปยังตำแหน่ง (ค่าตรงๆ ไม่ใช้ offset)"""
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   Sending: {cmd}")
        return self.send_command(cmd)
    
    def toggle_suction(self, state):
        self.send_command(f"DO(1,{1 if state else 0})")
        time.sleep(0.5)

    def pixel_to_robot(self, u, v):
        """แปลง pixel → robot coordinates"""
        if self.homography_matrix is None:
            return None, None
        point = np.array([u, v, 1], dtype=np.float32)
        result = np.dot(self.homography_matrix, point)
        return result[0] / result[2], result[1] / result[2]

print("✓ DobotControllerTCP class loaded")

✓ DobotControllerTCP class loaded


## 3️⃣ Configuration

In [10]:
# Robot IP
ROBOT_IP = '192.168.1.6'

# Camera ID
CAMERA_ID = 2

# Homography Matrix (จาก calibrate.py)
HOMOGRAPHY_MATRIX = np.array([
    [-0.004699341952697413, -0.3137634530917012, 84.66801170689713],
    [-0.4056710916298707, -0.013981190139846252, 174.27196033222742],
    [-2.645420242327231e-06, -0.00024026660997854653, 1.0],
], dtype=np.float32)

# Z Height (Online mode)
Z_HEIGHT = -50
R_ANGLE = 0

print("✓ Configuration loaded")
print(f"  Robot IP: {ROBOT_IP}")
print(f"  Camera ID: {CAMERA_ID}")
print(f"  Z Height: {Z_HEIGHT}")

✓ Configuration loaded
  Robot IP: 192.168.1.6
  Camera ID: 2
  Z Height: -50


## 4️⃣ Connect to Robot

In [12]:
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX)

if robot.connect(ROBOT_IP):
    print("\n" + "="*50)
    print(f"🤖 Connected to {ROBOT_IP}!")
    print("="*50)
else:
    print("⚠️ Failed to connect")

Clearing Errors...
Enabling Robot...

✅ Connected!

🤖 Connected to 192.168.1.6!


## 5️⃣ 🎯 Interactive Click & Move

**วิธีใช้:**
1. คลิกจุดที่ต้องการบนภาพกล้อง
2. Robot จะเคลื่อนที่ไปยังจุดที่คลิก
3. กด `Q` เพื่อออก

In [13]:
# =================================================================
# 🎯 Interactive Click & Move
# คลิกที่ภาพ → Robot เคลื่อนที่ไปยังจุดที่คลิก
# =================================================================

clicked_points = []  # เก็บจุดที่คลิก

def mouse_callback(event, x, y, flags, param):
    global clicked_points
    
    if event == cv2.EVENT_LBUTTONDOWN:
        # แปลง pixel → robot
        robot_x, robot_y = robot.pixel_to_robot(x, y)
        
        if robot_x is not None:
            clicked_points.append((x, y, robot_x, robot_y))
            print(f"\n📍 Clicked: Pixel({x}, {y}) → Robot({robot_x:.2f}, {robot_y:.2f})")
            print(f"   Moving to Robot position...")
            
            # เคลื่อนที่ไปยังจุดที่คลิก
            robot.move_to(robot_x, robot_y, Z_HEIGHT, R_ANGLE)

# เปิดกล้อง
cap = cv2.VideoCapture(CAMERA_ID)
if not cap.isOpened():
    print(f"❌ Cannot open camera {CAMERA_ID}")
else:
    print("="*50)
    print("🎯 Interactive Click & Move")
    print("="*50)
    print("คลิกที่ภาพ → Robot เคลื่อนที่ไปยังจุดที่คลิก")
    print("กด Q เพื่อออก")
    print("="*50)
    
    cv2.namedWindow('Click to Move Robot')
    cv2.setMouseCallback('Click to Move Robot', mouse_callback)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # วาดจุดที่คลิกไว้
        for i, (px, py, rx, ry) in enumerate(clicked_points[-5:]):  # แสดง 5 จุดล่าสุด
            cv2.circle(frame, (px, py), 8, (0, 0, 255), -1)
            cv2.circle(frame, (px, py), 12, (0, 0, 255), 2)
            cv2.putText(frame, f"({rx:.1f},{ry:.1f})", (px+15, py-5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # แสดงคำแนะนำ
        cv2.putText(frame, "Click to move robot | Q = Quit", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Z = {Z_HEIGHT}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        
        cv2.imshow('Click to Move Robot', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n" + "="*50)
    print(f"📊 Total clicks: {len(clicked_points)}")
    print("="*50)

🎯 Interactive Click & Move
คลิกที่ภาพ → Robot เคลื่อนที่ไปยังจุดที่คลิก
กด Q เพื่อออก

📍 Clicked: Pixel(325, 148) → Robot(38.09, 41.89)
   Moving to Robot position...
   Sending: MovJ(38.09098434448242,41.88505554199219,-50,0)

📍 Clicked: Pixel(328, 371) → Robot(-36.57, 39.59)
   Moving to Robot position...
   Sending: MovJ(-36.57125473022461,39.587989807128906,-50,0)

📍 Clicked: Pixel(157, 148) → Robot(38.89, 112.56)
   Moving to Robot position...
   Sending: MovJ(38.89237594604492,112.561767578125,-50,0)

📍 Clicked: Pixel(158, 378) → Robot(-38.16, 115.42)
   Moving to Robot position...
   Sending: MovJ(-38.15859603881836,115.4219970703125,-50,0)

📍 Clicked: Pixel(213, 218) → Robot(16.12, 89.56)
   Moving to Robot position...
   Sending: MovJ(16.120046615600586,89.55743408203125,-50,0)

📍 Clicked: Pixel(290, 255) → Robot(3.51, 56.57)
   Moving to Robot position...
   Sending: MovJ(3.5134921073913574,56.57156753540039,-50,0)

📍 Clicked: Pixel(197, 305) → Robot(-12.91, 97.27)
   Moving 

## 6️⃣ 🎯 Mark Multiple Points → Move Sequentially

**วิธีใช้:**
1. คลิก mark หลายจุดบนภาพ
2. กด `SPACE` เพื่อเริ่มให้ Robot วิ่งตามจุดที่ mark
3. กด `R` เพื่อ reset ล้างจุดทั้งหมด
4. กด `Q` เพื่อออก

In [ ]:
# =================================================================
# 🎯 Mark Multiple Points → Move Sequentially
# =================================================================

marked_points = []  # เก็บจุดที่ mark
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
colors = [(0,0,255), (0,255,0), (255,0,0), (0,255,255), 
          (255,0,255), (255,255,0), (128,0,255), (0,128,255)]

def mark_callback(event, x, y, flags, param):
    global marked_points
    
    if event == cv2.EVENT_LBUTTONDOWN and len(marked_points) < 8:
        robot_x, robot_y = robot.pixel_to_robot(x, y)
        
        if robot_x is not None:
            idx = len(marked_points)
            label = labels[idx]
            marked_points.append((label, x, y, robot_x, robot_y))
            print(f"✓ {label}: Pixel({x}, {y}) → Robot({robot_x:.2f}, {robot_y:.2f})")

# เปิดกล้อง
cap = cv2.VideoCapture(CAMERA_ID)
if not cap.isOpened():
    print(f"❌ Cannot open camera {CAMERA_ID}")
else:
    print("="*50)
    print("🎯 Mark Multiple Points Mode")
    print("="*50)
    print("คลิก = Mark จุด (สูงสุด 8 จุด)")
    print("SPACE = Robot วิ่งตามจุดที่ mark")
    print("R = Reset ล้างจุดทั้งหมด")
    print("Q = ออก")
    print("="*50)
    
    cv2.namedWindow('Mark Points')
    cv2.setMouseCallback('Mark Points', mark_callback)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # วาดจุดที่ mark ไว้
        for i, (label, px, py, rx, ry) in enumerate(marked_points):
            color = colors[i % len(colors)]
            cv2.circle(frame, (px, py), 8, color, -1)
            cv2.circle(frame, (px, py), 12, color, 2)
            cv2.putText(frame, f"{label}", (px+15, py+5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        
        # วาดเส้นเชื่อมจุด
        for i in range(1, len(marked_points)):
            pt1 = (marked_points[i-1][1], marked_points[i-1][2])
            pt2 = (marked_points[i][1], marked_points[i][2])
            cv2.line(frame, pt1, pt2, (255, 255, 255), 2)
        
        # แสดงคำแนะนำ
        cv2.putText(frame, f"Points: {len(marked_points)} | SPACE=Run | R=Reset | Q=Quit", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f"Z = {Z_HEIGHT}", (10, 55),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        
        cv2.imshow('Mark Points', frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
            
        elif key == ord('r'):
            marked_points = []
            print("\n🔄 Reset - ล้างจุดทั้งหมด")
            
        elif key == ord(' ') and len(marked_points) > 0:
            print("\n" + "="*50)
            print(f"🤖 Robot moving to {len(marked_points)} points...")
            print("="*50)
            
            for label, px, py, rx, ry in marked_points:
                print(f"\n📍 Moving to {label}: ({rx:.2f}, {ry:.2f})")
                robot.move_to(rx, ry, Z_HEIGHT, R_ANGLE)
                time.sleep(3)
            
            print("\n✅ Complete!")
    
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n" + "="*50)
    print("📊 Marked Points:")
    for label, px, py, rx, ry in marked_points:
        print(f"   {label}: Pixel({px}, {py}) → Robot({rx:.2f}, {ry:.2f})")
    print("="*50)

## 7️⃣ Home Robot

In [ ]:
print("⚠️ Robot will move to HOME!")
if input("Confirm? (y/n): ").lower() == 'y':
    robot.home()

## 8️⃣ Manual Test (Optional)

In [ ]:
# ทดสอบเคลื่อนที่ด้วยตนเอง
X, Y = 0, 0  # ใส่ค่า robot coordinates
Z = -50
R = -25

print(f"📍 Move to: ({X}, {Y}, {Z})")
if input("Confirm? (y/n): ").lower() == 'y':
    robot.move_to(X, Y, Z, R)
    print("✅ Done")